In [10]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.special as sc_special
from sklearn.metrics import mean_squared_error

In [11]:
def cuckoo_search(n, m, fit_func, lower_boundary, upper_boundary, iter_num=100, pa=0.25, beta=1.5, step_size=0.1):
    nests = generate_nests(n, m, lower_boundary, upper_boundary)
    fitness = calc_fitness(fit_func, nests)

    best_nest_index = np.argmax(fitness)
    best_fitness = fitness[best_nest_index]
    best_nest = nests[best_nest_index].copy()

    for _ in range(iter_num):
        nests = update_nests(fit_func, lower_boundary, upper_boundary, nests, best_nest, fitness, step_size)
        nests = abandon_nests(nests, lower_boundary, upper_boundary, pa)
        fitness = calc_fitness(fit_func, nests)
        
        max_nest_index = np.argmax(fitness)
        max_fitness = fitness[max_nest_index]
        max_nest = nests[max_nest_index]

        if (max_fitness > best_fitness):
            best_nest = max_nest.copy()
            best_fitness = max_fitness

    return best_nest, best_fitness

In [12]:
def generate_nests(n, m, lower_boundary, upper_boundary):
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    nests = np.empty((n, m))

    for each_nest in range(n):
        nests[each_nest] = lower_boundary + np.random.rand(m) * (upper_boundary - lower_boundary)

    return nests

In [13]:
def update_nests(fit_func, lower_boundary, upper_boundary, nests, best_nest, fitness, step_coefficient):
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    steps = levy_flight(n, m, 1.5)
    new_nests = nests.copy()

    for each_nest in range(n):
        step_size = step_coefficient * steps[each_nest] * (nests[each_nest] - best_nest)
        step_direction = np.random.rand(m)
        new_nests[each_nest] += step_size * step_direction
        # 更新部分：用 np.clip 限制新的巢穴位置
        new_nests[each_nest] = np.clip(new_nests[each_nest], lower_boundary, upper_boundary)

    new_fitness = calc_fitness(fit_func, new_nests)
    nests[new_fitness > fitness] = new_nests[new_fitness > fitness]
    
    return nests

In [14]:
def abandon_nests(nests, lower_boundary, upper_boundary, pa):
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    for each_nest in range(n):
        if np.random.rand() < pa:
            step_size = np.random.rand() * (nests[np.random.randint(0, n)] - nests[np.random.randint(0, n)])
            nests[each_nest] += step_size
            # 更新部分：用 np.clip 限制新的巢穴位置
            nests[each_nest] = np.clip(nests[each_nest], lower_boundary, upper_boundary)
    
    return nests

In [15]:
def levy_flight(n, m, beta):
    sigma_u = (sc_special.gamma(1+beta)*np.sin(np.pi*beta/2)/(sc_special.gamma((1+beta)/2)*beta*(2**((beta-1)/2))))**(1/beta)
    sigma_v = 1

    u =  np.random.normal(0, sigma_u, (n, m))
    v = np.random.normal(0, sigma_v, (n, m))

    steps = u/((np.abs(v))**(1/beta))

    return steps

In [16]:
def calc_fitness(fit_func, nests):
    n, m = nests.shape
    fitness = np.empty(n)

    for each_nest in range(n):
        fitness[each_nest] = fit_func(nests[each_nest])

    return fitness

In [17]:
def fit_func(nest):
    A, l_pos, epsilon_f_pos, c_s_pos_max, l_neg, epsilon_f_neg, c_s_neg_max = nest
    df_ex = pd.read_csv('data/data_ocv_ex.csv')
    df_sim = pd.read_csv('data/data_ocv_sim_ex.csv')

    v_t = df_ex['Voltage[V]']
    v_hat_t = df_sim['Voltage[V]']
    
    FF_V = mean_squared_error(v_t, v_hat_t)

    F = 96485.33212
    theta_0_pos = 0.915
    theta_0_neg = 0.008
    theta_100_pos = 0.254
    theta_100_neg = 0.855
    FF_C = np.abs((A * l_pos * epsilon_f_pos * F * c_s_pos_max * (theta_0_pos - theta_100_pos) / 3600) - 
                  (A * l_neg * epsilon_f_neg * F * c_s_neg_max * (theta_100_neg - theta_0_neg) / 3600))
    
    # 更新部分：归一化 FF_V 和 FF_C，并检查分母是否为零
    if np.max(v_t) != np.min(v_t):
        FF_V_norm = (FF_V - np.min(v_t)) / (np.max(v_t) - np.min(v_t))
    else:
        FF_V_norm = 0

    if np.max(FF_C) != np.min(FF_C):
        FF_C_norm = (FF_C - np.min(FF_C)) / (np.max(FF_C) - np.min(FF_C))
    else:
        FF_C_norm = 0
    
    w_Vn = 0.2
    w_c = 1 - w_Vn
    FF_M = w_Vn * FF_V_norm + w_c * FF_C_norm
    
    return FF_M

In [18]:
if __name__ == '__main__':
    lower_bounds = [0.378, 35, 0.35, 4.8, 35, 0.4, 2.9]
    upper_bounds = [0.395, 79, 0.5, 5.2, 79, 0.5, 3.3]

    best_nest, best_fitness = cuckoo_search(50, 7, fit_func, lower_bounds, upper_bounds, step_size=0.4)

    print('Max fitness: %.5f, Best nest position: (A: %.5f, l_pos: %.5f, epsilon_f_pos: %.5f, c_s_pos_max: %.5f, l_neg: %.5f, epsilon_f_neg: %.5f, c_s_neg_max: %.5f)' 
          % (best_fitness, best_nest[0], best_nest[1], best_nest[2], best_nest[3], best_nest[4], best_nest[5], best_nest[6]))

Max fitness: -0.29407, Best nest position: (A: 0.39409, l_pos: 51.94488, epsilon_f_pos: 0.39544, c_s_pos_max: 5.14199, l_neg: 47.91502, epsilon_f_neg: 0.45741, c_s_neg_max: 2.95426)


In [28]:
def fit_func(nest):
    A, l_pos, epsilon_f_pos, c_s_pos_max, l_neg, epsilon_f_neg, c_s_neg_max = nest
    df_ex = pd.read_csv('data/data_ocv_ex.csv')
    df_sim = pd.read_csv('data/data_ocv_sim_ex.csv')

    v_t = df_ex['Voltage[V]']
    v_hat_t = df_sim['Voltage[V]']
    
    FF_V = mean_squared_error(v_t, v_hat_t)

    F = 96485.33212
    theta_0_pos = 0.915
    theta_0_neg = 0.008
    theta_100_pos = 0.254
    theta_100_neg = 0.855
    FF_C = np.abs((A * l_pos * epsilon_f_pos * F * c_s_pos_max * (theta_0_pos - theta_100_pos) / 3600) - 
                  (A * l_neg * epsilon_f_neg * F * c_s_neg_max * (theta_100_neg - theta_0_neg) / 3600))
    
    # 更新部分：归一化 FF_V 和 FF_C，并检查分母是否为零
    if np.max(v_t) != np.min(v_t):
        FF_V_norm = (FF_V - np.min(v_t)) / (np.max(v_t) - np.min(v_t))
    else:
        FF_V_norm = 0

    if np.max(FF_C) != np.min(FF_C):
        FF_C_norm = (FF_C - np.min(FF_C)) / (np.max(FF_C) - np.min(FF_C))
    else:
        FF_C_norm = 0
    
    w_Vn = 0.8
    w_c = 1 - w_Vn
    FF_M = w_Vn * FF_V_norm + w_c * FF_C_norm
    
    return FF_M

In [29]:
if __name__ == '__main__':
    lower_bounds = [0.378, 35, 0.35, 4.8, 35, 0.4, 2.9]
    upper_bounds = [0.395, 79, 0.5, 5.2, 79, 0.5, 3.3]

    best_nest, best_fitness = cuckoo_search(50, 7, fit_func, lower_bounds, upper_bounds, step_size=0.4)

    print('Max fitness: %.5f, Best nest position: (A: %.5f, l_pos: %.5f, epsilon_f_pos: %.5f, c_s_pos_max: %.5f, l_neg: %.5f, epsilon_f_neg: %.5f, c_s_neg_max: %.5f)' 
          % (best_fitness, best_nest[0], best_nest[1], best_nest[2], best_nest[3], best_nest[4], best_nest[5], best_nest[6]))

Max fitness: -1.17627, Best nest position: (A: 0.37820, l_pos: 65.41416, epsilon_f_pos: 0.49169, c_s_pos_max: 4.88187, l_neg: 59.39536, epsilon_f_neg: 0.48402, c_s_neg_max: 3.12161)


In [20]:
# 加载数据
df_ex = pd.read_csv('data/data_ocv_ex.csv')
df_sim = pd.read_csv('data/data_ocv_sim_ex.csv')

v_t = df_ex['Voltage[V]']
v_hat_t = df_sim['Voltage[V]']

# 计算优化前后的MSE
initial_mse = mean_squared_error(v_t, v_hat_t)


In [26]:
l_pos = 51.94488e-6    #
epsilon_f_pos = 0.39544
c_s_pos_max = 5.14199e4
l_neg = 47.91502e-6    #
epsilon_f_neg = 0.45741
c_s_neg_max = 2.95426e4

In [22]:
param = pybamm.ParameterValues("OKane2022")
param

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x0000018FC7B42AC0>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x0000018FC7B43420>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x0000018FC7B43380>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current density for plating [A.m-2]': <function 

In [25]:
param.search("maximum")

Maximum concentration in negative electrode [mol.m-3]	33133.0
Maximum concentration in positive electrode [mol.m-3]	63104.0


In [32]:
# Create a simple DFN model
model = pybamm.lithium_ion.DFN()
# create experiment
experiment = pybamm.Experiment(
    ["Rest for 901 s",
        (
            "Discharge at 0.96 A for 147 s",
            "Rest for 361 s",
        ) * 123,
     "Discharge at 0.96 A for 134 s"
    ]
    
)
# Setting the current function）
param = pybamm.ParameterValues("OKane2022")
param["Negative electrode thickness [m]"] = l_pos
param["Positive electrode thickness [m]"] = l_neg
param["Negative electrode active material volume fraction"] = epsilon_f_neg
param["Positive electrode active material volume fraction"] = epsilon_f_pos
# param["Maximum concentration in negative electrode [mol.m-3]"] = c_s_neg_max
# param["Maximum concentration in positive electrode [mol.m-3]"] = c_s_pos_max


# Creating Simulation Objects
solver = pybamm.CasadiSolver(mode="fast")
simulation = pybamm.Simulation(model, parameter_values=param, solver=solver, experiment=experiment)

# Running simulations with time ranges consistent with data
solution = simulation.solve()  
# Getting results
time_opt = solution["Time [s]"].entries
# current = solution["Current [A]"].entries
voltage_opt = solution["Terminal voltage [V]"].entries

At t = 63.0824 and h = 1.89556e-14, the corrector convergence failed repeatedly or with |h| = hmin.
2024-07-15 13:53:46.367 - [ERROR] callbacks.on_experiment_error(224): Simulation error: Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDASolve returned "IDA_CONV_FAIL". Consult IDAS documentation.
